#Student Name: Martin Power
#Student ID: 9939245

## Etivity4 ##

### Task ###
With the final version of the code from the peer you selected in E-tivity 3 as a starting point, create a class that contains one tensor and provides the following methods:

* Calculate the size of the tensor as a 2D tuple
* Sum/subtract the tensor with another tensor of suitable size
* Multiply the tensor with a tensor of suitable size
* Calculate the determinant of the tensor, but only for tensors of size 2x2 (matrix)
* Calculate the inverse of the tensor, but only for tensors of size 2x2
* Calculate the cross product of the tensor with another tensor of suitable size

#### Notes ####
* Limit code to use tensors of rank 1 and rank 2 (vectors and matrices)
* Write your code without the use of imported modules.
* Provide suitable exception handling to prevent the above operations from being used on unsuitable tensors.
* Test your code with one example for each of the following scenarios:
  - sum of a 4x4 with a 4x4 matrix
  - multiplication of a 4x4 with a 4x4 matrix
  - multiplication of a 4x4 matrix with a suitable vector
  - determinant of a 2x2 matrix
  - inverse of a 2x2 matrix
  - cross-product of 2 suitable tensors

Code is pulled from Cormac Lavery (Commit 5635c9fb998a4005cf8e4f98a89575150676b802) and is available at the following [link](https://gitlab.com/CE4021/Group2/week3/blob/CormacLavery/Etivity3.ipynb)


In [195]:
# Reset to clear any stale variables
%reset -f

In [196]:
# Code from Etivity3 from Cormac Lavery ID:16139658

class Matrix:
    def __init__(self, row_list):
        self.row_list = row_list
        self.shape = self.__shape()

    def __shape(self):
        """MPP ADD DOC STRING"""
        rows = len(self.row_list)
        columns = len(self.row_list[0]) if rows > 0 else 0
        return rows, columns
    
    def __size__(self):
        return self.shape
    
    def size(self):
        return self.shape

    def __add__(self, other_matrix):
        return self.__add_sum_matrices(other_matrix, False)

    def __sub__(self, other_matrix):
        return self.__add_sum_matrices(other_matrix, True)

    def __mul__(self, other_matrix):
        row_count, column_count = self.shape
        row_count_other, column_count_other = other_matrix.shape
        if column_count != row_count_other:
            raise ValueError(
                'the product of 2 matrices can only be calculated if count columns of first matrix == count of rows '
                'rows in second matrix')
        product_matrix = []
        for i in range(row_count):
            matrix_entry = []
            for j in range(column_count_other):
                matrix_entry_value = 0
                for k in range(row_count_other):
                    matrix_entry_value += self.row_list[i][k] * other_matrix.row_list[k][j]
                matrix_entry.append(matrix_entry_value)
            product_matrix.append(tuple(matrix_entry))
        return Matrix(tuple(product_matrix))

    def __matmul__(self, other):
        self.__mul__(other)

    def __str__(self):
        return str(self.row_list)
    
    def __eq__(self, other_matrix):
        return self.row_list == other_matrix.row_list

    def __add_sum_matrices(self, other_matrix, subtract):
        row_count, column_count = self.shape
        if (row_count, column_count) != other_matrix.shape:
            raise ValueError("matrices may only be summed if same shape")

        row_list = []
        for i in range(row_count):
            col_list = []
            for j in range(column_count):
                if subtract:
                    new_value = self.row_list[i][j] - other_matrix.row_list[i][j]
                else:
                    new_value = self.row_list[i][j] + other_matrix.row_list[i][j]
                col_list.append(new_value)
            row_list.append(tuple(col_list))

        return Matrix(tuple(row_list))
    
    ##############################################################################
    ## New Functions Added for Etivity4
    ##############################################################################
    
    
    def det (self):
        """Calculates the determinant of a 2x2 matrix.
           NOTE: Matrices of higher dimensions are not supported
           """
        # Check size and shape
        row_count,column_count = self.shape
        if(row_count!=column_count):
            raise ValueError("Matrix must be square in order to calculate determinant")
        elif(row_count>2):
            raise ValueError("This Determinant function only support 2x2 matrices")

        return ((self.row_list[0][0]*self.row_list[1][1])-(self.row_list[0][1]*self.row_list[1][0]))


    def inv (self):
        """Calculates the inverse of a 2x2 matrix.
           NOTE: Matrices of higher dimensions are not supported
           """
        # Check size and shape
        row_count,column_count = self.shape
        if((row_count!=column_count)and(row_count!=2)):
            raise ValueError("Matrix must be 2x2 in order to calculate inverse using this function")
        
        # Get the determinant
        det = self.det()
        
        if(det==0):
            print("No inverse matrix exists for",self)
            return None
        else:
            a = self.row_list[1][1]/det
            b = -1*self.row_list[0][1]/det
            c = -1*self.row_list[1][0]/det
            d = self.row_list[0][0]/det
            return Matrix(((a, b), (c, d)))

    def cross_product(self, other):
        """ Calculates the cross product of 2 vectors.
            Returns a 3D vector of the form ((i,), (j,), (k,))
            where the values of i,j,k correspond to the vector
            coordinates in 3D space
        """
        row_count_s,column_count_s = self.shape
        row_count_o,column_count_o = other.shape
        
        if((row_count_s>3)or(row_count_o>3)):
            raise ValueError("Matrix exceeds 3 dimensions")
        elif((column_count_s!=1)or(column_count_o!=1)):
            raise ValueError("Vectors should only have 1 column")
   
        # If 1D or 2D vector passed to function, pad to 3 dimensions
        # before calculating cross product
        v = self.conv_to_vec()
        w = other.conv_to_vec()
        
        i = v[1]*w[2] - v[2]*w[1]; 
        j = v[2]*w[0] - v[0]*w[2];
        k = v[0]*w[1] - v[1]*w[0];
        
        return Matrix(((i,), (j,), (k,)))
    
    def conv_to_vec(self):
        """ Pads a 1D or 2D vector to a 3D vector
        """
        if(self.shape[0]==1):    # X Coord Only, Pad Y and Z
            return [self.row_list[0][0], 0, 0]
        elif(self.shape[0]==2):  # X and Y Coord Only, Pad Z
            return [self.row_list[0][0], self.row_list[1][0], 0]
        else:
            return [self.row_list[0][0], self.row_list[1][0], self.row_list[2][0]]

In [197]:
import unittest

##################################################
# Matrices
##################################################
# 2x2 Matrices
a = Matrix(((-2, 7), (9, 4)))        # A: 2x2 Matrix
b = Matrix(((4, 8), (-6, 1)))        # B: 2x2 Matrix
c = Matrix(((8, 2), ))               # C: 1x2 Row Vector
d = Matrix(((-3,), (4,)))            # D: 2x1 Column Vector

# 4x4 Matrices (Values from Cormac Lavery's Review)
e = Matrix(((4, 1, 3, 2), (3, 7, 4, 1), (9, 2, 8, 3), (1, 1, 5, 6)))
f = Matrix(((2, 6, 8, 9), (2, 6, 2, 4), (4, 3, 1, 6), (1, 1, 9, 2)))
g = Matrix(((1,), (2,), (3,), (4,)))
h = Matrix(((6, 7, 1, 3),))

# Cross Product Test Vectors
m = Matrix(((-3,), (4,), (5,)))
n = Matrix(((8,), (2,), (7,)))
o = Matrix(((-3,), (4,), ))
p = Matrix(((8,), (2,), ))
q = Matrix(((-3,), ))
r = Matrix(((8,), ))

# Basis Vectors i, j, k
i = Matrix(((1,), ))
j = Matrix(((0,), (1,), ))
k = Matrix(((0,), (0,), (1,)))

##################################################
# Unit Test
##################################################


class TestEtivity4(unittest.TestCase):
    

# Test multiplication of a 4x4 with a 4x4 matrix
# Test multiplication of a 4x4 matrix with a suitable vector
# Test determinant of a 2x2 matrix
# Test inverse of a 2x2 matrix
# Test cross-product of 2 suitable tensors    
    
    
    # Test sum of a 4x4 with a 4x4 matrix
    def test_001(self):
        # E+F
        expected_result = Matrix(((6, 7, 11, 11), (5, 13, 6, 5), (13, 5, 9, 9), (2, 2, 14, 8)))
        self.assertEqual(e+f, expected_result, "E+F Mismatch")
        
    
    # Test multiplication of a 4x4 matrix with a suitable vector
    # Scenarios:
    #  1 : 4x4 with 4x4 = 4x4
    #  2 : 4x4 with 4x1 = 4x1
    #  3 : 1x4 with 4x4 = 1x4
    #  4 : 4x4 with 2x2 = Error
    def test_002(self):
         # E*F
        expected_result = Matrix(((24, 41, 55, 62), (37, 73, 51, 81), (57, 93, 111, 143), (30, 33, 69, 55)))
        self.assertEqual(e*f, expected_result, "E*F Mismatch")

        # E*G (Matrix by Vector)
        expected_result = Matrix(((23,), (33,), (49,), (42,)))
        self.assertEqual(e*g, expected_result, "E*G Mismatch")
        
        # H*E (Vector by Matrix)
        expected_result = Matrix(((57, 60, 69,40), ))
        self.assertEqual(h*e, expected_result, "H*E Mismatch")
        
        # Check that E*A rasies a ValueError
        self.assertRaises(ValueError, lambda: e*a)
   
    # Test determinant of a 2x2 matrix
    def test_003(self):
         # det(A)
        expected_result = -71
        self.assertEqual(a.det(), expected_result, "Deterimant A Mismatch")
        
        # det(B)
        expected_result = 52
        self.assertEqual(b.det(), expected_result, "Deterimant B Mismatch")

    # Test inverse of a 2x2 matrix
    def test_004(self):
         # inverse(A)
        expected_result = Matrix(((-4/71, 7/71), (9/71, 2/71)))
        self.assertEqual(a.inv(), expected_result, "Inverse A Mismatch")
        
        # inverse(B)
        expected_result = Matrix(((1/52, -8/52), (6/52, 4/52)))
        self.assertEqual(b.inv(), expected_result, "Inverse B Mismatch")
        
    # Test cross-product of 2 suitable tensors
    # Scenarios:
    # 1 : 3D X 3D
    # 2 : 2D X 2D
    # 3 : Basis Vector Cross Product
    def test_005(self):
         # M X N
        expected_result = Matrix(((18,), (61,), (-38,)))
        self.assertEqual(m.cross_product(n), expected_result, "M X N Mismatch")
        
        # O X P
        expected_result = Matrix(((0,), (0,), (-38,)))
        self.assertEqual(o.cross_product(p), expected_result, "O X P Mismatch")
        
        # Q X R
        expected_result = Matrix(((0,), (0,), (0,)))
        self.assertEqual(q.cross_product(r), expected_result, "Q X R Mismatch")
        
        # I X J = K
        expected_result = Matrix(((0,), (0,), (1,)))
        self.assertEqual(i.cross_product(j), expected_result, "I X J Mismatch")
        
        # J X K = I
        expected_result = Matrix(((1,), (0,), (0,)))
        self.assertEqual(j.cross_product(k), expected_result, "J X K Mismatch")

        # K X I = J
        expected_result = Matrix(((0,), (1,), (0,)))
        self.assertEqual(k.cross_product(i), expected_result, "K X I Mismatch")


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.028s

OK
